# Setup

In [ ]:
from google import genai
from google.genai.types import GenerateContentConfig, SafetySetting, HarmBlockThreshold, HarmCategory, EmbedContentConfig
from IPython.display import Markdown, display
import numpy as np
from pypdf import PdfReader
from typing import *
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss

In [ ]:
!gcloud auth application-default login --project irn-77178-lab-ed

PROJECT_ID = "irn-77178-lab-ed"
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "europe-west1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

MODEL_ID = "gemini-2.0-flash-001"
TEXT_EMBEDDING_MODEL_ID = "text-embedding-005"

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=rT7sMEBap0ssMNDswgj9bxpbNI1kOP&access_type=offline&code_challenge=zBO_aFVPpeUaKOjd4A1vtlzsOUDVkxv20Nww4Y7lBlA&code_challenge_method=S256


Credentials saved to file: [C:\Users\p129811\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot add the project "irn-77178-lab-ed" to ADC as the quota project because the account in ADC does not have the "serviceusage.services.use" permission on this project. You might receive a "quota_exceeded" or "API not enabled" 

# Extraction PDF

### pip install

In [ ]:
!pip install --upgrade pypdf

  Obtaining dependency information for pdf2image from https://files.pythonhosted.org/packages/62/33/61766ae033518957f877ab246f87ca30a85b778ebaad65b7f74fa7e52988/pdf2image-1.17.0-py3-none-any.whl.metadata
  Obtaining dependency information for pytesseract from https://files.pythonhosted.org/packages/7a/33/8312d7ce74670c9d39a532b2c246a853861120486be9443eebf048043637/pytesseract-0.3.13-py3-none-any.whl.metadata


### travail

In [ ]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [4]:
old_doc = extract_text_from_pdf("ancien_clean.pdf")
new_doc = extract_text_from_pdf("nouveau.pdf")

print(old_doc[:1000])
print("#################")
print(new_doc[:1000])

Gynécologie
Obstétrique
Front matterChez le même éditeur
Dans la même collection
Activité physique et sportive : facteur de santé, par le Collège français des enseignants en médecine et trauma-
tologie du sport et de l'exercice physique (CFEMTSEP), 2019, 96 pages.
Anatomie et cytologie pathologiques, par le Collège français des pathologistes (CoPath), 3 e édition, 2019, 
416 pages.
Chirurgie maxillo-faciale et stomatologie, par le Collège hospitalo-universitaire français de chirurgie maxillofa -
ciale et stomatologie, 5e édition, 2021, 432 pages.
Dermatologie, par le Collège des enseignants en dermatologie de France (CEDEF), 7e édition, 2017, 472 pages.
Endocrinologie, diabétologie et maladies métaboliques, par le Collège des enseignants d'endocrinologie, dia -
bète et maladies métaboliques (CEEDMM), 5e édition, 2021, 568 pages.
Gériatrie, par le Collège national des enseignants de gériatrie (CNEG), 5e édition, 2021, 400 pages.
Gynécologie obstétrique, par le Collège national des gynéc

### saves

In [12]:
with open("saves/ancien.txt", "w") as f:
    f.write(old_doc)
with open("saves/nouveau.txt", "w") as f:
    f.write(new_doc)

# Splitting en chunks

### pip install

In [8]:
!pip install langchain tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/6f/07/c67ad1724b8e14e2b4c8cca04b15da158733ac60136879131db05dda7c30/tiktoken-0.9.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
   ---------------------------------------- 10.2/893.9 kB ? eta -:--:--
   --- ----------------------------------- 71.7/893.9 kB 975.2 kB/s eta 0:00:01
   ----------------- ---------------------- 399.4/893.9 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 893.9/893.9 kB 6.3 MB/s eta 0:00:00


### travail

In [ ]:
def split_text_into_chunks(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    chunks = []
    # Une approche simple par caractères, mais vous pouvez améliorer avec un split par paragraphe ou phrase
    words = text.split()
    current_chunk = []
    current_length = 0

    for word in words:
        if current_length + len(word) + 1 <= chunk_size:
            current_chunk.append(word)
            current_length += len(word) + 1
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = current_chunk[-overlap:] + [word] # Garder un chevauchement
            current_length = sum(len(w) + 1 for w in current_chunk)

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

def split_text_langchain(text: str, chunk_size: int = 250, overlap: int = 25) -> List[str]:
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
    )
    return text_splitter.split_text(text)

In [10]:
chunks_old = split_text_langchain(old_doc)
chunks_new = split_text_langchain(new_doc)

print(f"Nombre de chunks dans l'ancien document: {len(chunks_old)}")
print(f"Premier chunk de l'ancien document: {chunks_old[0]}")
print(f"Nombre de chunks dans le nouveau document: {len(chunks_new)}")
print(f"Premier chunk du nouveau document: {chunks_new[0]}")

Nombre de chunks dans l'ancien document: 3295
Premier chunk de l'ancien document: Gynécologie
Obstétrique
Front matterChez le même éditeur
Dans la même collection
Activité physique et sportive : facteur de santé, par le Collège français des enseignants en médecine et trauma-
tologie du sport et de l'exercice physique (CFEMTSEP), 2019, 96 pages.
Anatomie et cytologie pathologiques, par le Collège français des pathologistes (CoPath), 3 e édition, 2019, 
416 pages.
Chirurgie maxillo-faciale et stomatologie, par le Collège hospitalo-universitaire français de chirurgie maxillofa -
ciale et stomatologie, 5e édition, 2021, 432 pages.
Dermatologie, par le Collège des enseignants en dermatologie de France (CEDEF), 7e édition, 2017, 472 pages.
Nombre de chunks dans le nouveau document: 3279
Premier chunk du nouveau document: Gynécologie 
ObstétriqueGynécologie 
Obstétrique
Sous l’égide du Collège National des Gynécologues et Obstétriciens Français 
et du Collège des Enseignants de Gynécologie-Ob

### saves

In [13]:
import pickle

with open("saves/chunks/ancien_chunks.pkl", "wb") as f:
    pickle.dump(chunks_old, f)
with open("saves/chunks/nouveau_chunks.pkl", "wb") as f:
    pickle.dump(chunks_new, f)

# Encoding

### travail

In [ ]:
old_embeddings, new_embeddings = [], []
i, j = 0, 0

while i < len(chunks_old):
    old_embeddings_curr = client.models.embed_content(
        model=TEXT_EMBEDDING_MODEL_ID,
        contents=chunks_old[i:i+50],
        config=EmbedContentConfig(output_dimensionality=128),
    ).embeddings
    i += 50
    old_embeddings.extend(old_embeddings_curr)

while j < len(chunks_new):
    new_embeddings_curr = client.models.embed_content(
        model=TEXT_EMBEDDING_MODEL_ID,
        contents=chunks_new[j:j+50],
        config=EmbedContentConfig(output_dimensionality=128),
    ).embeddings
    j += 50
    new_embeddings.extend(new_embeddings_curr)

In [51]:
print(old_embeddings[0].values)
print("###########################")
print(new_embeddings[0].values)

[0.07292214035987854, 0.028604796156287193, 0.00985381007194519, 0.008197312243282795, -0.08715558797121048, 0.09809369593858719, -0.04790036752820015, 0.005290105938911438, 0.030218008905649185, -0.021994443610310555, -0.08133779466152191, -0.0002647455839905888, -0.022741608321666718, -0.0755920484662056, -0.024394595995545387, 0.017677146941423416, -0.0025116787292063236, -0.03381466120481491, -0.011464131064713001, 0.03316674008965492, 0.025264890864491463, -0.07628980278968811, -0.08097600936889648, -0.0016912149731069803, -0.008765246719121933, 0.012538186274468899, -0.06853815913200378, 0.012070356868207455, -0.03066873364150524, 0.01292774174362421, 0.02784457802772522, 0.02827315405011177, 0.03347572311758995, -0.00963573157787323, 0.05163216590881348, -0.005396260414272547, 0.04904455691576004, -0.006912999786436558, 0.03427436575293541, 0.04417436569929123, 0.053925156593322754, -0.005296215880662203, -0.02003784105181694, 0.01724827289581299, -0.0062719229608774185, 0.02379

### saves

In [52]:
with open("saves/embeddings/ancien_embeddings.pkl", "wb") as f:
    pickle.dump(old_embeddings, f)
with open("saves/embeddings/nouveau_embeddings.pkl", "wb") as f:
    pickle.dump(new_embeddings, f)

# Vectorisation

### version cheap

In [63]:
old_vectors = {chunks_old[i]: old_embeddings[i].values for i in range(len(chunks_old))}
new_vectors = {chunks_new[i]: new_embeddings[i].values for i in range(len(chunks_new))}

### travail

In [71]:
np_old_embeddings = np.array([embed.values for embed in old_embeddings])
np_new_embeddings = np.array([embed.values for embed in new_embeddings])

# Créer des index FAISS pour chaque document
dimension_embedding = np_old_embeddings.shape[1]

old_index = faiss.IndexFlatL2(dimension_embedding)
old_index.add(np_old_embeddings)

new_index = faiss.IndexFlatL2(dimension_embedding)
new_index.add(np_new_embeddings)

### saves

In [74]:
with open("saves/vectors_cheap/ancien_vecteurs_cheap.pkl", "wb") as f:
    pickle.dump(old_vectors, f)
with open("saves/vectors_cheap/nouveau_vecteurs_cheap.pkl", "wb") as f:
    pickle.dump(new_vectors, f)

with open("saves/faiss_indexes/old_indexes.pkl", "wb") as f:
    pickle.dump(old_index, f)
with open("saves/faiss_indexes/new_indexes.pkl", "wb") as f:
    pickle.dump(new_index, f)

# Comparaison & identification des différences

In [107]:
def find_differences(chunks_old: List[str], embeddings_old: np.ndarray, index_old: faiss.IndexFlatL2,
                     chunks_new: List[str], embeddings_new: np.ndarray, index_new: faiss.IndexFlatL2,
                     similarity_threshold: float = 0.9) -> List[str]:
    differences = []

    # Parcourir les chunks du document 2 et chercher leur correspondance dans le document 1
    for i, chunk_new in enumerate(chunks_new):
        query_embedding = embeddings_new[i].reshape(1, -1)
        distances, indices = index_old.search(query_embedding, k=1) # Chercher le plus proche dans doc1

        closest_distance = distances[0][0]
        closest_chunk_old_index = indices[0][0]

        # Si la distance est grande (faible similarité), ou si le chunk n'est pas "trouvé" (distance > seuil),
        # cela indique une possible différence ou un ajout.
        # Note: L2 distance plus faible = plus similaire.
        # Vous devrez ajuster le seuil en fonction de vos observations.
        if closest_distance > (1 - similarity_threshold): # Adapter la condition pour L2 distance
            differences.append(f"Ajout/Modification probable dans nouveau doc: \n'{chunk_new}'\n---------------\n(Distance au plus proche de l'ancien doc: {closest_distance:.4f})\nChunk comparé de l'ancien doc:\n---------------\n'{chunks_old[closest_chunk_old_index]}")
        else:
            # Pour des modifications subtiles, vous pouvez aussi comparer le chunk original avec le chunk trouvé
            # si la similarité est juste au-dessus du seuil mais pas parfaite.
            # Cela nécessiterait une étape supplémentaire d'analyse.
            pass # Ici, le chunk est considéré comme similaire

    # Vous pouvez aussi faire l'inverse (chercher les différences de doc1 par rapport à doc2)
    # selon votre besoin de détection des suppressions.

    return differences

In [108]:
differences = find_differences(chunks_old, np_old_embeddings, old_index, chunks_new, np_new_embeddings, new_index)

print(f"{len(differences)} différences notables ont été trouvées !")
print("\n##################\n")

for difference in differences:
    print(difference)
    print("\n##################\n")

16 différences notables ont été trouvées !

##################

Ajout/Modification probable dans nouveau doc: 
'vention chirurgicale en urgence.
Traitement
Le médecin discute avec Madame Dupont de la nécessité de l’intervention chirurgicale et des risques 
associés. Il lui propose également de désigner une personne de confiance pour l’accompagner dans le 
processus de prise de décision et lui apporter un soutien émotionnel. Madame Dupont choisit sa meil-
leure amie comme personne de confiance. Ensemble, elles discutent des implications de cette décision. 
Finalement, Madame Dupont décide de suivre les recommandations médicales et d’avoir l’intervention 
chirurgicale. Sa meilleure amie reste à ses côtés tout au long du processus, lui apportant un réconfort 
précieux et l’assurance qu’elle a pris la meilleure décision pour sa santé.'
---------------
(Distance au plus proche de l'ancien doc: 0.1174)
Chunk comparé de l'ancien doc:
---------------
'prise en charge chirurgicale urgente en ca

# Agent synthétisation différences

In [ ]:
safety_settings = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
]

differences_text = "\n##################\n".join(differences)
prompt = f"""
Voici une liste de différences potentielles détectées entre deux documents.
Veuillez analyser ces différences et fournir une liste claire et concise de chaque ajout ou modification significative,
en expliquant brièvement de quoi il s'agit.
Différences détectées :
{differences_text}

Liste des ajouts/modifications:
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(safety_settings=safety_settings)
).text

display(Markdown(response))

with open("RESULT.md", 'w', encoding='utf-8') as f:
    f.write(response)

Voici une liste claire et concise des ajouts ou modifications significatives détectées, expliquant brièvement de quoi il s'agit.

*   **Ajout d'un récit de cas de Madame Dupont :** Ajout d'un paragraphe décrivant la prise en charge d'une patiente (Madame Dupont) nécessitant une intervention chirurgicale en urgence, mettant l'accent sur l'importance de la personne de confiance et la prise de décision éclairée.
*   **Ajout d'un scénario clinique GEU (Grossesse Extra-Utérine) :** Ajout d'un scénario clinique concernant une prise en charge chirurgicale en urgence d'une patiente avec suspicion de GEU, incluant la description de l'intervention et des explications à la patiente.
*   **Ajout d'une définition de la GEU et ses localisations:** Ajout d'une section définissant la Grossesse Extra-Utérine (GEU), et décrivant les différentes localisations possibles de la nidation.
*   **Ajout des Signes cliniques de choc et de la conduite à tenir face à une anémie mal tolérée:** Ajout d'une liste de signes cliniques de choc, et de la conduite à tenir face à une anémie mal tolérée.
*   **Ajout d'une vignette clinique sur l'évaluation des pertes sanguines:** Ajout d'une introduction à une vignette clinique concernant une patiente de 28 ans qui consulte pour aménorrhée et désir de conception et présentant des signes d'hyperandrogénie.
*   **Ajout d'informations sur un enregistrement tococardiographique :** Ajout d'informations sur les éléments à vérifier lors d'un enregistrement tococardiographique (antécédents, groupe sanguin, etc.).
*   **Ajout d'un récit de cas de mastite puerpérale :** Ajout d'un récit de cas clinique concernant une patiente présentant une mastite puerpérale, avec fièvre et abcès du sein.
*   **Ajout d'informations sur la prise en charge d'un abcès du sein:** Ajout d'informations sur l'interrogatoire, le bilan préopératoire, et la prise en charge anesthésique d'une patiente devant être opérée pour un abcès du sein.
*   **Ajout d'un tableau sur les traitements compatibles ou non avec l'allaitement:** Ajout d'un tableau listant des médicaments (antiémétiques, antihypertenseurs) et leur compatibilité avec l'allaitement.
*   **Ajout d'un tableau sur les psychotropes et l'allaitement:** Ajout d'un tableau listant des psychotropes et leur compatibilité avec l'allaitement.
*   **Ajout d'items concernant la prise en charge du diabète de type 1 et 2 :** Ajout d'items concernant la prise en charge du diabète, l'hypoglycémie, et les complications comme la cétoacidose et le coma hyperosmolaire.
*   **Ajout d'un scénario de dystocie des épaules :** Ajout d'un scénario clinique concernant un accouchement compliqué d'une dystocie des épaules, avec des conséquences pour le nouveau-né.
*   **Ajout d'un récit de cas de suspicion de pré-éclampsie :** Ajout d'un récit de cas clinique concernant une femme enceinte présentant des signes de pré-éclampsie.
*   **Ajout d'un récit de cas concernant Valentine O. :** Ajout d'un récit de cas d'une jeune patiente de 17 ans souhaitant initier une contraception.
*   **Ajout de questions réponses concernant la conduite à tenir en cas d'infection urinaire :** Ajout de questions réponses concernant la conduite à tenir en cas d'infection urinaire.